In [72]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np

# Sklearn utilities
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error as mse
from sklearn.compose import make_column_selector, make_column_transformer

# Sklearn models
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.tree import DecisionTreeRegressor

Directory layout of data:
```
data
|--- vgchartz
     |--- vgchartz-6_23_2020.csv
     |--- vgchartz-7_7_2020.csv
|--- vgsales
     |
     |--- vgsales.csv
```

# Loading data

In [3]:
# Read in data
vgchartz1 = pd.read_csv('/content/data/vgchartz/vgchartz-6_23_2020.csv')
vgchartz2 = pd.read_csv('/content/data/vgchartz/vgchartz-7_7_2020.csv')
vgchartz = pd.concat([vgchartz1, vgchartz2])

vgsales = pd.read_csv('/content/data/vgsales/vgsales.csv')

In [4]:
vgchartz.head()

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,3,/games/boxart/full_3351915AmericaFrontccc.jpg,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,42.06,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-24
1,4,/games/boxart/full_5741036AmericaFrontccc.jpg,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,NaN,35.05,NaN,NaN,NaN,NaN,NaN,2005-03-22,2020-03-04
2,5,/games/boxart/full_6662824AmericaFrontccc.png,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,NaN,22.00,NaN,NaN,NaN,NaN,NaN,2001-10-16,2020-02-03
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
4,7,/games/boxart/full_6800951AmericaFrontccc.jpg,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,NaN,20.00,NaN,NaN,NaN,NaN,NaN,1981-10-23,2020-02-26


In [5]:
vgchartz.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115857 entries, 0 to 57949
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     115857 non-null  int64  
 1   img            115857 non-null  object 
 2   title          115857 non-null  object 
 3   console        115857 non-null  object 
 4   genre          115857 non-null  object 
 5   publisher      115857 non-null  object 
 6   developer      115823 non-null  object 
 7   vg_score       1964 non-null    float64
 8   critic_score   13087 non-null   float64
 9   user_score     772 non-null     float64
 10  total_shipped  4745 non-null    float64
 11  total_sales    38638 non-null   float64
 12  na_sales       25800 non-null   float64
 13  jp_sales       13986 non-null   float64
 14  pal_sales      26222 non-null   float64
 15  other_sales    30880 non-null   float64
 16  release_date   109100 non-null  object 
 17  last_update    23483 non-null   obj

In [6]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [7]:
vgsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


## Clean up vgchartz

In [8]:
vgchartz.head()

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,3,/games/boxart/full_3351915AmericaFrontccc.jpg,Warriors,Series,Action,Unknown,Omega Force,NaN,NaN,NaN,42.06,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-24
1,4,/games/boxart/full_5741036AmericaFrontccc.jpg,God of War,Series,Action,Sony Computer Entertainment,SIE Santa Monica Studio|Ready at Dawn|Javagrou...,NaN,NaN,NaN,35.05,NaN,NaN,NaN,NaN,NaN,2005-03-22,2020-03-04
2,5,/games/boxart/full_6662824AmericaFrontccc.png,Devil May Cry,Series,Action,Capcom,Capcom|Ninja Theory,NaN,NaN,NaN,22.00,NaN,NaN,NaN,NaN,NaN,2001-10-16,2020-02-03
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
4,7,/games/boxart/full_6800951AmericaFrontccc.jpg,Frogger,Series,Action,Konami,Konami|Parker Brothers|SCE Cambridge Studio|Bl...,NaN,NaN,NaN,20.00,NaN,NaN,NaN,NaN,NaN,1981-10-23,2020-02-26


- probably get rid of the "Unnamed: 0" column
- parse the "Developer" column to split any names separated by | into a list(?) or maybe just select out the first one?

In [9]:
vgsales = vgsales.dropna(subset=['Year', 'Publisher', 'Name', 'Global_Sales'])
vgsales['Year'] = vgsales['Year'].astype(int)

# Initial attempts at modelling

## Basic attempt, no feature engineering or modifications

In [ ]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
vgsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


(16327, 11)

In [ ]:
# Drop entries with NA for Global_Sales and convert to int
vgsales = vgsales.dropna(subset=['Year'])
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.shape

# Test train split
X = vgsales[['Name', 'Platform', 'Genre', 'Publisher']]
y = vgsales['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape


((12245, 4), (12245,))

In [ ]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.21573120874981172

In [ ]:
baseline = np.mean(y_train)
baseline_preds = np.ones(y_train.shape) * baseline
baseline_mse = mse(y_train, baseline_preds)
baseline_mse

2.060874554500041

In [ ]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.9940113286090706

In [ ]:
# See the RMSE of both the train_mse and baseline_mse
np.sqrt(train_mse), np.sqrt(test_mse), np.sqrt(baseline_mse)

(0.46446873818354206, 1.9985022713544938, 1.4355746426083322)

These RMSE values should be in the natural units of the prediction target (sales numbers in millions). The train RMSE was okay, but the test RMSE was somehow worse than the baseline. I wonder if this has to do with the fact that I haven't accounted for the increase in overall video game sales over time.

## Initial attempt at scaling global sales as % of total annual sales

In [ ]:
vgsales['Year'].dropna().shape

(16327,)

In [ ]:
# Calculate the total video game sales for each year in the dataset, and put into a dict
vgsales['Year'] = vgsales['Year']
min_year = vgsales['Year'].min()
max_year = vgsales['Year'].max()
min_year, max_year

(1980, 2020)

In [ ]:
sales_per_year = {}
for y in range(min_year, max_year+1):
  # Pull out all records from the vgsales df from that year, and sum up their Global_Sales column
  relevant_records = vgsales[vgsales['Year']==y]
  total = relevant_records['Year'].sum()
  sales_per_year[int(y)] = total

sales_per_year

{1980: 17820,
 1981: 91126,
 1982: 71352,
 1983: 33711,
 1984: 27776,
 1985: 27790,
 1986: 41706,
 1987: 31792,
 1988: 29820,
 1989: 33813,
 1990: 31840,
 1991: 81631,
 1992: 85656,
 1993: 119580,
 1994: 241274,
 1995: 436905,
 1996: 524948,
 1997: 577133,
 1998: 757242,
 1999: 675662,
 2000: 698000,
 2001: 964482,
 2002: 1659658,
 2003: 1552325,
 2004: 1529052,
 2005: 1886705,
 2006: 2022048,
 2007: 2412414,
 2008: 2867424,
 2009: 2874879,
 2010: 2530590,
 2011: 2290529,
 2012: 1321884,
 2013: 1099098,
 2014: 1172148,
 2015: 1237210,
 2016: 693504,
 2017: 6051,
 2018: 0,
 2019: 0,
 2020: 2020}

In [ ]:
# Now create a new column for the df containing the fraction of global sales for the year that this video game comprised
global_sales_frac = []
def calc_sales_pct(year, sales):
  total_sales = sales_per_year[year]
  return sales / total_sales
vgsales['Global_Sales_Pct'] = vgsales.apply(lambda x: calc_sales_pct(x[3], x[10]), raw=True, axis=1)
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Global_Sales_Pct
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.000041
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.001448
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,0.000012
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.000011
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.000060


In [ ]:
vgsales['Global_Sales_Pct'].mean(), vgsales['Global_Sales_Pct'].std()

(-2.7852474483154534e-17, 1.000030625526383)

In [ ]:
# Standardize the Global_Sales_Pct column
# Save these two values so we can un-standardize later
original_global_pct_mean, original_global_pct_std = np.mean(vgsales['Global_Sales_Pct']), np.std(vgsales['Global_Sales_Pct'])
vgsales['Global_Sales_Pct'] = (vgsales['Global_Sales_Pct'] - np.mean(vgsales['Global_Sales_Pct'])) / np.std(vgsales['Global_Sales_Pct'])
vgsales['Global_Sales_Pct'].mean(), vgsales['Global_Sales_Pct'].std()

(-2.7852474483154534e-17, 1.000030625526383)

In [ ]:
# Now try fitting the same kind of LinearRegression() pipeline from before, but using this Global_Sales_Pct column instead of Global_Sales
X_pct = vgsales[['Name', 'Platform', 'Genre', 'Publisher', 'Year']] # Including Year so we can select it out for later use
y_pct = vgsales['Global_Sales_Pct']

X_pct_train, X_pct_test, y_pct_train, y_pct_test = train_test_split(X_pct, y_pct, test_size=0.25, random_state=42)
X_pct_train.shape, y_pct_train.shape


((12245, 5), (12245,))

In [ ]:
# Select out Year from the X datasets and set aside for later use in expanding our % predictions into sales numbers
X_pct_train_year, X_pct_test_year = X_pct_train['Year'], X_pct_test['Year']
X_pct_train, X_pct_test = X_pct_train.drop(columns=['Year']), X_pct_test.drop(columns=['Year'])

In [ ]:
# Set up a pipeline that uses OHE and then runs linear regression
pct_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pct_pipe.fit(X_pct_train, y_pct_train)
pct_train_preds = pct_pipe.predict(X_pct_train)
pct_train_mse = mse(y_pct_train, pct_train_preds)
pct_train_mse

0.10058570196817558

In [ ]:
pct_test_preds = pct_pipe.predict(X_pct_test)
pct_test_mse = mse(y_pct_test, pct_test_preds)
pct_test_mse

2.32172649971741

An improvement over the earlier model, by a factor of $(3.9940113286090706 - 2.32172649971741) / 3.9940113286090706 = 0.4186980685$

Lets see what happens if we use these predicted %s to calculate the actual sales numbers for their respective years.

In [ ]:
X_pct_test.head()

,Name,Platform,Genre,Publisher
5468,The Sims 2,GC,Simulation,Electronic Arts
8769,The Terminator: Dawn of Fate,XB,Action,Atari
4577,NHL 2K6,PS2,Sports,Take-Two Interactive
2345,F1 2010,X360,Racing,Codemasters
11065,Super Robot Wars OG: The Moon Dwellers,PS4,Misc,Namco Bandai Games


In [ ]:
# Un-standardize the y_pct_test column, as well as the pct_test_preds series
y_pct_test_unstd = y_pct_test * original_global_pct_std + original_global_pct_mean
pct_test_preds_unstd = pct_test_preds * original_global_pct_std + original_global_pct_mean
y_pct_test_unstd.mean(), y_pct_test_unstd.std(), pct_test_preds_unstd.mean(), pct_test_preds_unstd.std()

(9.95544145561234e-07,
 7.91927495774232e-06,
 2.312248410634095e-06,
 2.76587556826986e-05)

In [ ]:
# Concat the unstandardized y_test with the year values
y_test_expanded = pd.concat([y_pct_test_unstd, X_pct_test_year], axis=1)
y_test_expanded.head()

,Global_Sales_Pct,Year
5468,1.749081e-07,2005
8769,9.038007e-08,2002
4577,2.226103e-07,2005
2345,3.516966e-07,2010
11065,1.297757e-07,2016


In [ ]:
# Reset indeces so that we can match up the pct_test_preds with the years accurately
y_pct_test_unstd1 = y_pct_test_unstd.reset_index()
X_pct_test_year1 = X_pct_test_year.reset_index()
pct_test_preds_unstd1 = pd.Series(pct_test_preds_unstd).reset_index()

In [ ]:
pct_preds_expanded = pd.concat([pct_test_preds_unstd1, X_pct_test_year1], axis=1).drop(columns=['index'])
pct_preds_expanded.head()

,0,Year
0,3.323800e-07,2005
1,-4.833353e-07,2002
2,3.880485e-07,2005
3,4.561527e-07,2010
4,3.069853e-07,2016


In [ ]:
pd.Series(pct_test_preds_unstd).reindex(y_pct_test_unstd.index)

,0
5468,NaN
8769,NaN
4577,NaN
2345,1.086709e-07
11065,NaN
...,...
14456,NaN
12760,NaN
12819,NaN
15424,NaN


In [ ]:
X_test.head()

,Name,Platform,Genre,Publisher
5468,The Sims 2,GC,Simulation,Electronic Arts
8769,The Terminator: Dawn of Fate,XB,Action,Atari
4577,NHL 2K6,PS2,Sports,Take-Two Interactive
2345,F1 2010,X360,Racing,Codemasters
11065,Super Robot Wars OG: The Moon Dwellers,PS4,Misc,Namco Bandai Games


In [ ]:
def reconstr_global_sales(year, sales_pct):
  total_sales = sales_per_year[year]
  return total_sales * sales_pct

y_test_expanded['Global_Sales_Reconstructed'] = y_test_expanded.apply(lambda x: reconstr_global_sales(x[1], x[0]), raw=True, axis=1)

In [ ]:
y_test_expanded.head()

,Global_Sales_Pct,Year,Global_Sales_Reconstructed
5468,1.749081e-07,2005,0.33
8769,9.038007e-08,2002,0.15
4577,2.226103e-07,2005,0.42
2345,3.516966e-07,2010,0.89
11065,1.297757e-07,2016,0.09


In [ ]:
pct_preds_expanded['Global_Sales_Reconstructed'] = pct_preds_expanded.apply(lambda x: reconstr_global_sales(x[1], x[0]), raw=True, axis=1)
pct_preds_expanded.head()

,0,Year,Global_Sales_Reconstructed
0,3.323800e-07,2005,0.627103
1,-4.833353e-07,2002,-0.802171
2,3.880485e-07,2005,0.732133
3,4.561527e-07,2010,1.154335
4,3.069853e-07,2016,0.212896


In [ ]:
expanded_pct_mse = mse(y_test_expanded['Global_Sales_Reconstructed'], pct_preds_expanded['Global_Sales_Reconstructed'])
expanded_pct_mse

586.2665902528116

Okay, so this did not work nearly as well as I thought it might LOL

## Try modelling with 'Year' as a feature

In [ ]:
# Read in dataset from scratch again
vgsales = pd.read_csv('/content/data/vgsales/vgsales.csv')

# Drop entries with NA for Global_Sales and convert to int
vgsales = vgsales.dropna(subset=['Year'])
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.shape

# Test train split
X = vgsales[['Name', 'Platform', 'Genre', 'Publisher', 'Year']]
y = vgsales['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape


((12245, 5), (12245,))

In [ ]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.18606597859500074

In [ ]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

4.942023745188282

In [ ]:
baseline = np.mean(y_train)
baseline_preds = np.ones(y_train.shape) * baseline
baseline_mse = mse(y_train, baseline_preds)
baseline_mse

2.060874554500041

Wow, I think this is overfitting a lot. The train_mse went down, but the test_mse is huge (4.9 vs 3.9 with the original basic model).

## Try ridge regression + Year feature

In [ ]:
# Set up a pipeline that uses OHE and then runs ridge regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('ridge', Ridge())])
grid = GridSearchCV(pipe, param_grid={'ridge__alpha':list(np.linspace(0.1, 10.0, 50))}) # np.linspace(0.1, 10.0, 0.4)
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.9567877014965808

In [ ]:
grid.best_params_

{'ridge__alpha': 2.928571428571429}

# Examining the data more closely

Going to look at features like the publisher and the game title, to see if I can find anything interesting. I also want to investigate the feasibility + usefulness of adding a "Franchise" column to indicate what franchise a game belongs to (eg. Pokemon, Super Mario, etc), if any.

## Publisher data cleaning

In [10]:
vgsales = vgsales.dropna(subset=['Global_Sales', 'Publisher'])

In [11]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [12]:
unique_publishers = vgsales['Publisher'].unique()
unique_publishers

array(['Nintendo', 'Microsoft Game Studios', 'Take-Two Interactive',
       'Sony Computer Entertainment', 'Activision', 'Ubisoft',
       'Bethesda Softworks', 'Electronic Arts', 'Sega', 'SquareSoft',
       'Atari', '505 Games', 'Capcom', 'GT Interactive',
       'Konami Digital Entertainment',
       'Sony Computer Entertainment Europe', 'Square Enix', 'LucasArts',
       'Virgin Interactive', 'Warner Bros. Interactive Entertainment',
       'Universal Interactive', 'Eidos Interactive', 'RedOctane',
       'Vivendi Games', 'Enix Corporation', 'Namco Bandai Games',
       'Palcom', 'Hasbro Interactive', 'THQ', 'Fox Interactive',
       'Acclaim Entertainment', 'MTV Games', 'Disney Interactive Studios',
       'Majesco Entertainment', 'Codemasters', 'Red Orb', 'Level 5',
       'Arena Entertainment', 'Midway Games', 'JVC', 'Deep Silver',
       '989 Studios', 'NCSoft', 'UEP Systems', 'Parker Bros.', 'Maxis',
       'Imagic', 'Tecmo Koei', 'Valve Software', 'ASCII Entertainment',
     

In [13]:
[p for p in list(unique_publishers) if 'Ubisoft' in p]

['Ubisoft', 'Ubisoft Annecy']

In [14]:
[p for p in list(unique_publishers) if 'Sony' in p]

['Sony Computer Entertainment',
 'Sony Computer Entertainment Europe',
 'Sony Online Entertainment',
 'Sony Computer Entertainment America',
 'Sony Music Entertainment']

There might be a benefit from consolidating all the Ubisoft and Sony ones respectively.

In [15]:
def pub_mapper(name):
  if 'Sony' in name:
    return 'Sony'
  elif 'Ubisoft' in name:
    return 'Ubisoft'
  return name
vgsales['Publisher_Consolidated'] = vgsales.apply(lambda x: pub_mapper(x[5]), axis=1, raw=True)
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Publisher_Consolidated
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo


In [16]:
# Double check that we didn't drop any records or end up with missing data
vgsales.Publisher_Consolidated.isna().sum()

0

In [17]:
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.shape

# Test train split
X = vgsales[['Name', 'Platform', 'Genre', 'Publisher_Consolidated']]
y = vgsales['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape

((12218, 4), (12218,))

In [18]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.25312920273603734

In [19]:
baseline = np.mean(y_train)
baseline_preds = np.ones(y_train.shape) * baseline
baseline_mse = mse(y_train, baseline_preds)
baseline_mse

2.0670350951072165

In [20]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.7028251162152594

Slight improvement in the test mse (3.9 -> 3.7)

In [21]:
coefs = pipe.named_steps['linreg'].coef_
for c, f in zip(coefs, pipe.feature_names_in_):
  print(f + ": " + str(c))

Name: -0.08536352726928416
Platform: -0.6729875031363535
Genre: -0.6129875042147555
Publisher_Consolidated: -0.6829875029566145


It appears that the most influential factors are the publisher, genre and platform. Name is very unimportant. This makes sense, although I think this does suggest that any info about game franchises is not being captured (naturally). This makes me think that it might be worth incorporating a "Franchise" feature after all.

## Adding a "Franchise" feature

In [22]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Publisher_Consolidated
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo


In [23]:
vgsales['Name'].isna().sum()

0

In [24]:
unique_names = vgsales['Name'].unique()
unique_names

array(['Wii Sports', 'Super Mario Bros.', 'Mario Kart Wii', ...,
       'Plushees', 'Woody Woodpecker in Crazy Castle 5', 'Know How 2'],
      dtype=object)

In [25]:
# Manually construct a list of some of the most popular franchises (from https://en.wikipedia.org/wiki/List_of_best-selling_video_game_franchises)
franchises = ['Pokemon',
              'Fire Emblem',
              'Wii Sports',
              'Mario Kart',
              'Super Mario',
              'Mario',
              'Kirby',
              'Metroid',
              'Tetris',
              'Grand Theft Auto',
              'Call of Duty',
              'FIFA',
              'Minecraft',
              'Final Fantasy',
              "Assassin's Creed",
              "Sonic",
              "Resident Evil",
              "Zelda",
              "NBA",
              "Madden",
              "Star Wars",
              "Monster Hunter"
              ]
for f in franchises:
  ct = len([n for n in unique_names if f in n])
  print(f"{f}: {ct}")

Pokemon: 35
Fire Emblem: 13
Wii Sports: 3
Mario Kart: 9
Super Mario: 29
Mario: 97
Kirby: 24
Metroid: 14
Tetris: 21
Grand Theft Auto: 12
Call of Duty: 20
FIFA: 41
Minecraft: 2
Final Fantasy: 67
Assassin's Creed: 15
Sonic: 51
Resident Evil: 30
Zelda: 22
NBA: 85
Madden: 25
Star Wars: 53
Monster Hunter: 16


Note: should experiment with whether we get better results by breaking down into specific Mario sub-franchises, or treating them all as a monolith of "Mario"

In [26]:
# Use this function with df.apply to create a new column of franchises
def add_franchise(title):
  for f in franchises:
    if f in title:
      return f
  return ""

In [27]:
vgsales['Franchise'] = vgsales.apply(lambda x: add_franchise(x[1]), raw=True, axis=1)
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Publisher_Consolidated,Franchise
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,Wii Sports
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo,Super Mario
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo,Mario Kart
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,Wii Sports
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo,Pokemon


In [28]:
# Test train split
X = vgsales[['Name', 'Platform', 'Genre', 'Publisher_Consolidated', 'Franchise']]
y = vgsales['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape

((12218, 5), (12218,))

In [29]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.25312863305223354

In [30]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.559659605009973

Another slight improvement in test MSE (3.7 -> 3.5), but honestly less of an improvement than I was hoping for! I thought this would add a lot of information to predict with, since franchises are quite important to consumers.

In [31]:
coefs = pipe.named_steps['linreg'].coef_
for c, f in zip(coefs, pipe.feature_names_in_):
  print(f + ": " + str(c))

Name: -0.07333687651894566
Platform: -0.4932301115465472
Genre: -0.4332301139422435
Publisher_Consolidated: -0.503230111147262
Franchise: -0.4932301115465472


## Add a feature for title length
Erik thought this might work so I'll give it a try

In [69]:
vgsales['Title_Length'] = vgsales.apply(lambda x: len(x[1]), raw=True, axis=1)
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Publisher_Consolidated,Franchise,Title_Length
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,Wii Sports,10
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo,Super Mario,17
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo,Mario Kart,14
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,Wii Sports,17
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo,Pokemon,24


In [70]:
# Test train split
X = vgsales[['Name', 'Platform', 'Genre', 'Publisher_Consolidated', 'Franchise', 'Title_Length']]
y = vgsales['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape

((12218, 6), (12218,))

In [74]:
col_transformer = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=object)), remainder='passthrough')

In [75]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('transformer', col_transformer), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.25324100931782695

In [76]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.512856154117167

In [79]:
coefs = pipe.named_steps['linreg'].coef_
for c, f in zip(coefs, pipe.feature_names_in_):
  print(f + ": " + str(c))

Name: -0.12316302708065506
Platform: -0.5006217946460411
Genre: -0.43113618799910425
Publisher_Consolidated: -0.448960978739633
Franchise: -0.4863924008179338
Title_Length: -0.21754016319840846


## Try using decision tree regression...?

In [34]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('tree', DecisionTreeRegressor())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.00027152561794074317

In [35]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.7995115639577706

Not unexpected, considering the extreme overfitting as evident from the low training MSE. Lets try some methods for regularizing the tree.

In [40]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('tree', DecisionTreeRegressor())])
grid = GridSearchCV(pipe, param_grid={'tree__max_depth':list(range(3, 8)), 'tree__min_samples_leaf':list(range(1, 5))})
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

1.2986543019064851

In [41]:
test_preds = grid.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.304504196068465

Hey, I think this is our best test MSE so far (without the dataset restriction by year that I tried later in this notebook).

In [42]:
grid.best_params_

{'tree__max_depth': 7, 'tree__min_samples_leaf': 4}

In [43]:
# Lets try again with even wider grid search spaces, since both of the best_params_ values were at the upper bounds of what we checked
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('tree', DecisionTreeRegressor())])
grid = GridSearchCV(pipe, param_grid={'tree__max_depth':list(range(3, 15)), 'tree__min_samples_leaf':list(range(1, 9))})
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

1.2465923366118004

In [44]:
test_preds = grid.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

3.1381423952241447

In [45]:
grid.best_params_

{'tree__max_depth': 14, 'tree__min_samples_leaf': 5}

Seems that there's still some room to explore with the max tree depth, but at least we seem to have found a good stopping point for the min_samples_leaf at 5.

I wonder how the accuracy looks if we combine the tree approach with the dataset restriction. Lets go to the next section!

# Exploring restricting the target value range
This is related to my attempts to scale the global scales as a % of total global sales from each year. As a quick and rough proxy for that approach, I want to try modelling on a subset of the years where sales were approximately all in the same range. I've identified 2002-2015 (inclusive) as a good range for this.

In [ ]:
sales_per_year

{1980: 17820,
 1981: 91126,
 1982: 71352,
 1983: 33711,
 1984: 27776,
 1985: 27790,
 1986: 41706,
 1987: 31792,
 1988: 29820,
 1989: 33813,
 1990: 31840,
 1991: 81631,
 1992: 85656,
 1993: 119580,
 1994: 241274,
 1995: 436905,
 1996: 524948,
 1997: 577133,
 1998: 757242,
 1999: 675662,
 2000: 698000,
 2001: 964482,
 2002: 1659658,
 2003: 1552325,
 2004: 1529052,
 2005: 1886705,
 2006: 2022048,
 2007: 2412414,
 2008: 2867424,
 2009: 2874879,
 2010: 2530590,
 2011: 2290529,
 2012: 1321884,
 2013: 1099098,
 2014: 1172148,
 2015: 1237210,
 2016: 693504,
 2017: 6051,
 2018: 0,
 2019: 0,
 2020: 2020}

In [46]:
# Keep the modifications from previous steps: using Year, Publisher_Consolidated, and Franchise

vgsales_subset = vgsales[(vgsales['Year'] >= 2002) & (vgsales['Year'] <= 2015)]

# Test train split
X = vgsales_subset[['Name', 'Platform', 'Genre', 'Publisher_Consolidated', 'Franchise']]
y = vgsales_subset['Global_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape

((9855, 5), (9855,))

In [47]:
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('linreg', LinearRegression())])
pipe.fit(X_train, y_train)
train_preds = pipe.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.23301612291355323

In [48]:
test_preds = pipe.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

2.8678004550454985

Relatively big improvement in test MSE! 3.55 -> 2.87. I think this indicates that there is promise in the idea of accounting for total annual sales for the year when predicting a game's sales.

Alternatively... could try excluding results before 2002? But I wonder if that would hurt some franchises more than others (eg ones that had especially influential entries early on?)

In [49]:
# Calculate train and test RMSE
np.sqrt(train_mse), np.sqrt(test_mse)

(0.4827174358913848, 1.6934581350141191)

The test RMSE went down from 1.99 to 1.69. Not a *massive* improvement, but nontrivial I suppose? (it's a factor of 300,000 since these values are in millions)

## Using decision tree regression on restricted dataset

In [57]:
# Lets try again with even wider grid search spaces, since both of the best_params_ values were at the upper bounds of what we checked
# Set up a pipeline that uses OHE and then runs linear regression
pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')), ('tree', DecisionTreeRegressor())])
grid = GridSearchCV(pipe, param_grid={'tree__max_depth':list(range(17, 30)), 'tree__min_samples_leaf':list(range(1, 6))})
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
train_mse = mse(y_train, train_preds)
train_mse

0.9247366391141939

In [58]:
test_preds = grid.predict(X_test)
test_mse = mse(y_test, test_preds)
test_mse

2.8611594615093314

Not much variation in the test MSE, still hovering around 2.86.



In [60]:
grid.best_params_

{'tree__max_depth': 28, 'tree__min_samples_leaf': 3}

It seems we finally found a good stopping point for tree max_depth, in addition to min_samples_leaf!

Next time I pick this up, I should look into the other tree regularization methods to see if any of them can help. Or maybe I should back off on the regularization so it can be a bit better at predicting?


In [61]:
np.sqrt(train_mse), np.sqrt(test_mse)

(0.9616322785317649, 1.6914962197738816)

In [62]:
y_train.mean()

0.48615930999492646

If the mean target value is 0.48, then I feel like these RMSEs are actually not that great...? I'm surprised though, I thought the video game sales in this band of years were relatively high compared to the range of the full dataset. Idk...

In [65]:
unique_titles = vgsales['Name'].unique()
unique_titles

array(['Wii Sports', 'Super Mario Bros.', 'Mario Kart Wii', ...,
       'Plushees', 'Woody Woodpecker in Crazy Castle 5', 'Know How 2'],
      dtype=object)

In [66]:
lens = [len(t) for t in unique_titles]
lens

[10,
 17,
 14,
 17,
 24,
 6,
 21,
 8,
 25,
 9,
 10,
 13,
 27,
 7,
 12,
 18,
 18,
 29,
 17,
 44,
 29,
 16,
 19,
 27,
 29,
 27,
 43,
 22,
 30,
 39,
 23,
 19,
 25,
 26,
 30,
 20,
 23,
 27,
 12,
 6,
 36,
 14,
 14,
 18,
 41,
 34,
 19,
 12,
 19,
 14,
 21,
 33,
 12,
 20,
 11,
 13,
 23,
 6,
 17,
 12,
 14,
 30,
 19,
 9,
 25,
 14,
 27,
 16,
 7,
 9,
 6,
 13,
 31,
 14,
 10,
 13,
 34,
 15,
 18,
 24,
 7,
 38,
 20,
 28,
 30,
 36,
 38,
 19,
 35,
 26,
 13,
 38,
 10,
 26,
 8,
 25,
 23,
 12,
 9,
 25,
 12,
 7,
 28,
 25,
 30,
 15,
 13,
 14,
 26,
 7,
 17,
 19,
 7,
 27,
 19,
 20,
 20,
 23,
 14,
 23,
 12,
 20,
 36,
 17,
 6,
 13,
 14,
 12,
 35,
 20,
 40,
 16,
 14,
 17,
 15,
 46,
 33,
 15,
 8,
 12,
 7,
 19,
 17,
 16,
 19,
 18,
 28,
 19,
 25,
 15,
 36,
 18,
 9,
 15,
 16,
 17,
 14,
 40,
 14,
 45,
 10,
 23,
 30,
 38,
 41,
 14,
 26,
 18,
 9,
 26,
 14,
 15,
 35,
 14,
 13,
 22,
 12,
 16,
 24,
 32,
 9,
 64,
 26,
 13,
 17,
 24,
 14,
 30,
 17,
 7,
 8,
 23,
 24,
 26,
 37,
 18,
 11,
 39,
 35,
 41,
 26,
 8,
 15,
 8,
 13,
 

In [67]:
np.mean(lens)

24.744635761589404

In [68]:
min(lens), max(lens)

(1, 132)